In [70]:
import pythreejs
from pythreejs import *

from numpy import cos, sin, pi
import time
import ipywidgets

In [102]:
class hand:
    def __init__(self, x=0, y=0, z=100, r=10, side=1, phase=0):
        self.x=x
        self.y=y
        self.z=z
        self.r=r
        self.side = side
        self.phase = phase
        self.view =  Mesh(
            SphereBufferGeometry(2, 32, 16),
            MeshStandardMaterial(color="white")
        )
        self.view.position = self.position(0)

    def position(self, t):
        alpha = pi * (t + self.phase) 
        return self.x+self.side*self.r*cos(alpha), self.z-self.r*sin(alpha), self.y
    
    def update(self, t):
        self.view.position = self.position(t)
    
class ball:
    def __init__(self, color, launch_hand, reception_hand, time_of_departure, flight_duration, height=50 ):
        self.launch_hand = launch_hand
        self.reception_hand = reception_hand
        self.time_of_departure = time_of_departure
        self.flight_duration = flight_duration
        self.time_in_hand = 1
        self.height = height
        self.view = Mesh(
            SphereBufferGeometry(5, 32, 16),
            MeshStandardMaterial(color=color)
        )
        self.view.position = self.position(0)
    
    def position(self, t):
        t1 = (t-self.time_of_departure) % (2*self.flight_duration)
        if t1 < self.flight_duration:
            launch, reception = self.launch_hand, self.reception_hand
            time_flying = t1
        else:
            launch, reception = self.reception_hand, self.launch_hand
            time_flying = t1 - self.flight_duration
        if time_flying < self.time_in_hand:
            return launch.position(t)
        else:
            x0,z0,y0 = launch   .position(t-time_flying+self.time_in_hand)
            x1,z1,y1 = reception.position(t-time_flying+self.flight_duration)
            a = (time_flying-self.time_in_hand)/(self.flight_duration-self.time_in_hand)
            x = x0 * (1-a) + x1 * a
            y = y0 * (1-a) + y1 * a
            z = z0 * (1-a) + z1 * a + self.height * 4*a*(1-a)
            return x, z, y
    
    def update(self, t):
        self.view.position = self.position(t)

In [103]:
left_hand  = hand(x=-25, side=-1)
right_hand = hand(x= 25, side=1, phase=1)

b1 = ball("blue", left_hand, right_hand, time_of_departure=0, flight_duration=3)
b2 = ball("yellow", right_hand, left_hand, time_of_departure=1, flight_duration=3)
b3 = ball("red",  left_hand, right_hand, time_of_departure=2, flight_duration=3)
def update(t):
    left_hand.update(t)
    right_hand.update(t)
    b1.update(t)
    b2.update(t)
    b3.update(t)

scene = Scene(children=[left_hand.view, right_hand.view, b1.view,b2.view,b3.view],
              camera = PerspectiveCamera( position=[0, 100,-400]) #, aspect=view_width/view_height)
             )

scene

Preview(child=Scene(children=(Mesh(geometry=SphereBufferGeometry(heightSegments=16, radius=2.0, widthSegments=…

In [104]:
s = ipywidgets.FloatSlider(min=0, max=10)
s.observe(lambda change: update(s.value))
s


FloatSlider(value=0.0, max=10.0)

In [67]:
t = 0;
dt = 1./24
while True:
    update(t)
    t = t + dt
    time.sleep(dt)

KeyboardInterrupt: 